In [72]:
# imports 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as stat
import os
plt.style.use('fivethirtyeight')


In [73]:
df = pd.read_csv("/Users/daniellancet/Desktop/Edu_Org_Eco/Data/cleaned.csv")

In [74]:

# Scholarships, PTGS, K-12, Charter, Fundraising
scholarship_ntee = ["B82", "B825", "B824"]
PTGs_ntee = ["B94"]
P_12_ntee = ["B2", "B20", "B21", "B24", "B25", "B28", "B29"] # merged with charters? 
fundraising_ntee = ["B11", "B112", "B113", "B114", "B115", "B116", "B117", "B118", "B119", "B12", "B122", "B123", "B124","B125", "B126", "B127", "B128", "B129"]
fundraising_main_ntee = ["B12", "B122",  "B123",  "B124",  "B125",  "B126", "B127"]
k_12 = ["B20", "B2"]
B29_charters = ["B29"]



In [75]:
def count_range(low, high, series):
    count = np.sum((series > low) & (series <= high))
    return count

def percents(df, main_category,  column, intervals, year = 2019):  
    pd.options.display.float_format = '{:.2f}'.format
    if main_category != 'all':
        df = df[df["Main_Category"] == main_category]
    else:
        df = df

    f_series = df[column].dropna()
    total = len(f_series)

    counts = []
    for interval in intervals: 
        low, high = interval
        count = count_range(low, high, f_series)
        counts.append(count)
    counts = np.array(counts)
    
    assert(np.sum(counts) == total)
   
    percents_lst = np.round((counts/np.sum(counts))*100, 2)
    return percents_lst

intervals = [(-np.inf, 25000), (25000, 100000), (100000, 500000), (500000, 1000000), (1000000, np.inf)]
intervals_fundraising = [(-np.inf, 0), (0, 10000), (10000, 25000), (25000, 100000), (100000, 500000), (500000, 1000000), (1000000, np.inf)]
intervals_PTG = [
    (-np.inf, 0),
    (0, 10000),
    (10000, 25000),
    (25000, 50000),
    (50000, 100000),
    (100000, 150000),
    (150000, np.inf)
]


In [76]:
#categories = ["Parent Teacher Groups", "Scholarship", "Fundraising", "Fraternities", "Education Services"]
categories = ["all", "Parent Teacher Groups", "Scholarship", "Single Support Fundraising", "Fraternities", "Education Services"]


In [77]:
pd.options.display.float_format = '{:.0f}'.format

def finances_descriptions(df, main_categories, column): 
    describe_tables = []
    for category in main_categories:
        
        if category == 'all':
            temp_df = df
        else:
            temp_df = df[df["Main_Category"] == category]

        if category == 'Single Support Fundraiser':
            print(temp_df[column].describe())

        median = temp_df[column].median()
        mean = temp_df[column].mean()
        std = temp_df[column].std()
        minimum = temp_df[column].min()
        maximum = temp_df[column].max()
        missing = temp_df[column].isna().sum()
        count = len(temp_df[column])

        describe_table = pd.Series(
            {
                "Median": median,
                "Mean": mean,
                "Standard Deviation": std,
                "Minimum": minimum,
                "Maximum": maximum,
                "Missing": missing,
                "Total": count
            },
            name=category
        )
        
        describe_tables.append(describe_table)
        
    finance_df = pd.DataFrame(describe_tables).T
    finance_df.columns = main_categories
    return finance_df
    
finances_descriptions(df, categories, 'totrev')

,all,Parent Teacher Groups,Scholarship,Single Support Fundraising,Fraternities,Education Services
Median,0,0,0,0,0,0
Mean,2489503,27517,319148,1688951,89004,628500
Standard Deviation,73131326,276918,5925149,33788850,1689196,13454331
Minimum,-21573356,-44602,-8021699,-2616127,-432636,-1974000
Maximum,11078595000,33424349,444544338,2811160094,218813192,1115319662
Missing,47728,4144,8899,2795,6232,4906
Total,233897,36173,30163,28319,28120,20470


In [78]:
df_totrev = finances_descriptions(df, categories, 'totrev').to_csv("describe_totrev.csv")
df_total_assets = finances_descriptions(df, categories, 'total_assets').to_csv("describe_total_assets.csv")
df_total_income = finances_descriptions(df, categories, 'total_income').to_csv("describe_total_income.csv")

In [79]:
percents_lst = []
rev_ranges = [
    "Under 25,000",
    "25,000 - 100,000",
    "100,001 - 500,000",
    "500,001 - 1,000,000",
    "Above 1,000,000"
]
for cat in categories:
    percents_lst.append(percents(df, cat,  "totrev", intervals))

df_percents = pd.DataFrame(np.array(percents_lst)).T
df_percents.index = rev_ranges
df_percents.columns = categories



In [80]:
df_percents.to_csv("percent_ranges.csv")